In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import gc

print('loading files...')
base_path = "input/"
train = pd.read_csv(base_path+'train.csv', na_values=-1)
test = pd.read_csv(base_path+'test.csv', na_values=-1)
col_to_drop = train.columns[train.columns.str.startswith('ps_calc_')]
train = train.drop(col_to_drop, axis=1)  
test = test.drop(col_to_drop, axis=1)  

for c in train.select_dtypes(include=['float64']).columns:
    train[c]=train[c].astype(np.float32)
    test[c]=test[c].astype(np.float32)
for c in train.select_dtypes(include=['int64']).columns[2:]:
    train[c]=train[c].astype(np.int8)
    test[c]=test[c].astype(np.int8)    

print(train.shape, test.shape)

# custom objective function (similar to auc)

def gini(y, pred):
    g = np.asarray(np.c_[y, pred, np.arange(len(y)) ], dtype=np.float)
    g = g[np.lexsort((g[:,2], -1*g[:,1]))]
    gs = g[:,0].cumsum().sum() / g[:,0].sum()
    gs -= (len(y) + 1) / 2.
    return gs / len(y)

def gini_xgb(pred, y):
    y = y.get_label()
    return 'gini', gini(y, pred) / gini(y, y)

def gini_lgb(preds, dtrain):
    y = list(dtrain.get_label())
    score = gini(y, preds) / gini(y, y)
    return 'gini', score, True

# xgb
params = {'eta': 0.02, 'max_depth': 4, 'subsample': 0.9, 'colsample_bytree': 0.9, 
          'objective': 'binary:logistic', 'eval_metric': 'auc', 'silent': True}

X = train.drop(['id', 'target'], axis=1)
features = X.columns
X = X.values
y = train['target'].values
sub=test['id'].to_frame()
sub['target']=0

sub_train = train['id'].to_frame()
sub_train['target']=0

nrounds=10**6  # need to change to 2000
kfold = 5  # need to change to 5
skf = StratifiedKFold(n_splits=kfold, random_state=0)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(' xgb kfold: {}  of  {} : '.format(i+1, kfold))
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    d_train = xgb.DMatrix(X_train, y_train) 
    d_valid = xgb.DMatrix(X_valid, y_valid) 
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    xgb_model = xgb.train(params, d_train, nrounds, watchlist, early_stopping_rounds=100, 
                          feval=gini_xgb, maximize=True, verbose_eval=100)
    sub['target'] += xgb_model.predict(xgb.DMatrix(test[features].values), 
                        ntree_limit=xgb_model.best_ntree_limit+50) / (kfold)
    
    sub_train['target'] += xgb_model.predict(xgb.DMatrix(train[features].values), 
                        ntree_limit=xgb_model.best_ntree_limit+50) / (kfold)
    
gc.collect()
sub.head(2)

sub.to_csv(base_path+'test_sub_xgb.csv', index=False, float_format='%.5f')
sub_train.to_csv(base_path+'train_sub_xgb.csv', index=False, float_format='%.5f')


# lgb
sub['target']=0
sub_train['target']=0

params = {'metric': 'auc', 'learning_rate' : 0.01, 'max_depth':8, 'max_bin':10,  'objective': 'binary', 
          'feature_fraction': 0.8,'bagging_fraction':0.9,'bagging_freq':5,  'min_data': 500}

skf = StratifiedKFold(n_splits=kfold, random_state=1)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(' lgb kfold: {}  of  {} : '.format(i+1, kfold))
    X_train, X_eval = X[train_index], X[test_index]
    y_train, y_eval = y[train_index], y[test_index]
    lgb_model = lgb.train(params, lgb.Dataset(X_train, label=y_train), nrounds, 
                  lgb.Dataset(X_eval, label=y_eval), verbose_eval=100, 
                  feval=gini_lgb, early_stopping_rounds=100)
    sub['target'] += lgb_model.predict(test[features].values, 
                        num_iteration=lgb_model.best_iteration) / (kfold)
    sub_train['target'] += lgb_model.predict(train[features].values, 
                        num_iteration=lgb_model.best_iteration) / (kfold)
    
sub.to_csv(base_path+'test_sub_lgb.csv', index=False, float_format='%.5f') 
sub_train.to_csv(base_path+'train_sub_lgb.csv', index=False, float_format='%.5f')

gc.collect()
sub.head(2)

C:\Users\Crossbell\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


loading files...
(595212, 39) (892816, 38)
 xgb kfold: 1  of  5 : 
Will train until valid error hasn't decreased in 100 rounds.
[0]	train-gini:0.202082	valid-gini:0.205887
[100]	train-gini:0.248736	valid-gini:0.240129
[200]	train-gini:0.277074	valid-gini:0.258586
[300]	train-gini:0.297776	valid-gini:0.270673
[400]	train-gini:0.312015	valid-gini:0.276940
[500]	train-gini:0.322203	valid-gini:0.279789
[600]	train-gini:0.330076	valid-gini:0.281123
[700]	train-gini:0.337154	valid-gini:0.281701
[800]	train-gini:0.343712	valid-gini:0.281872
[900]	train-gini:0.350425	valid-gini:0.281703
[1000]	train-gini:0.356594	valid-gini:0.282228
[1100]	train-gini:0.362406	valid-gini:0.281569
Stopping. Best iteration:
[1014]	train-gini:0.357331	valid-gini:0.282276

 xgb kfold: 2  of  5 : 
Will train until valid error hasn't decreased in 100 rounds.
[0]	train-gini:0.201049	valid-gini:0.198643
[100]	train-gini:0.249840	valid-gini:0.234957
[200]	train-gini:0.275303	valid-gini:0.257248
[300]	train-gini:0.296437

,id,target
0,0,0.025786
1,1,0.026809
